In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten
from keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard

In [2]:
classnames=['COVID','NORMAL','Viral_Pneumonia']

CNN MODEL


In [3]:
model = Sequential()

model.add(Conv2D(32,(5,5), activation='relu', input_shape = (299,299,3)))
model.add(MaxPool2D())
model.add(Conv2D(64,(3,3), activation='relu',))
# model.add(MaxPool2D())
# # model.add(Dropout(rate=0.25))

# model.add(Conv2D(128,(3,3), activation='relu',))
# model.add(MaxPool2D())
# model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64, activation= 'relu'))
# model.add(Dropout(rate=0.50))
model.add(Dense(units= 3, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',optimizer = "adam",metrics = ['acc'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 295, 295, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 147, 147, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 145, 145, 64)      18496     
                                                                 
 flatten (Flatten)           (None, 1345600)           0         
                                                                 
 dense (Dense)               (None, 64)                86118464  
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                        

In [5]:

# Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [6]:
training_set = train_datagen.flow_from_directory('DATASET/TRAIN',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',classes=classnames)

Found 3000 images belonging to 3 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'DATASET/TEST',
        target_size=(299, 299),
        batch_size=32,
        class_mode='categorical',classes=classnames)

Found 700 images belonging to 3 classes.


In [8]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
lr = 1e-4
num_epochs = 100
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

In [10]:
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard()]
history=model.fit_generator(training_set, 
        epochs=10,
        validation_data = test_set,
        callbacks=callbacks)

C:\Users\CSE AI\.conda\envs\tf\lib\site-packages\ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == "__main__":


Epoch 1/10
 3/94 [..............................] - ETA: 2:09 - loss: 0.2878 - acc: 0.8854

KeyboardInterrupt: 

In [ ]:
print(history.history.keys())


In [ ]:
import matplotlib.image as mpimg

In [ ]:
print(history.history.keys())
import matplotlib.pyplot as plt
# summarize model for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
path ="Normal-42.png"
img = image.load_img(path, target_size=(299,299))
img = image.img_to_array(img)/255
img = np.array([img])
img.shape

In [ ]:
path ="Normal-"
for i in range(41, 50) :
    path = "Normal-" + str(i) + ".png"
    img = image.load_img(path, target_size=(299,299))
    img = image.img_to_array(img)/255
    img = np.array([img])
    predictions=(model.predict(img))
    print(predictions)
    predictions = (predictions>0.5).astype("int32")
    if predictions==1:
      print("NORMAL")
    else:
      print("SOME COVID SYMPTOMS ARE DETECTED IN PATIENTS XRAY")

In [ ]:
predictions = (model.predict(img)>0.5).astype("int32")
if predictions==1:
  print("NORMAL")
else:
  print("SOME COVID SYMPTOMS ARE DETECTED IN PATIENTS XRAY")